In [50]:
import os
import h5py
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import keras.backend as K
import random

In [51]:
class RunParameters:
    '''classe che gestisce i parametri della run'''
    
    def __init__(self, out_dir: str, ndir=0, ntoy=100):
        '''init: imposto il nome della cartella di output, genero il nome della cartella contenente il file e cerco il file'''
        
        self.out_dir = out_dir
        self.folder_name = os.listdir(self.out_dir+'/0/')[ndir]
        self.fname = [name for name in os.listdir(self.out_dir+'/0/'+self.folder_name) if f'_toy{ntoy}_t.txt' in name][0]
        print('\nFolder name: ' + self.folder_name)
        print('File name: ' + self.fname)
        
        return
    
    def fetch_parameters(self) -> list:
        '''uso il nome del file e della cartella per estrarre i parametri della run'''
        
        self.check_point_t = int(self.fname.split("patience", 1)[1].split("_", 1)[0])
        self.toys = int(self.fname.split("toy", 1)[1].split("_", 1)[0])
        self.ref = (self.fname.split("_")[2]).split("ref")[1]
        self.bkg = (self.fname.split("_")[3]).split("bkg")[1]
        self.sig = (self.fname.split("_")[4]).split("sig")[1]
        self.w_clip = self.folder_name.split('wclip',1)[1]
        self.latent = int(self.folder_name.split("latent", 1)[1].split("_", 1)[0])
        self.layers = int(self.folder_name.split("layers", 1)[1].split("_", 1)[0])
        self.epochs = int(self.folder_name.split('epochs',1)[1].split('_')[0])
        
        self.parameters = [self.toys, self.w_clip, self.epochs, self.check_point_t, self.ref, self.bkg, self.sig, self.latent, self.layers]
        
        return self.parameters
    
    def print_parameters(self):
        '''stampo i parametri per controllare corrispondano alla run'''
        
        print(f'\nParameters:                                              \
                        \n Toys:          {self.toys}                      \
                        \n Latent space:  {self.latent}                    \
                        \n Layers:        {self.layers}                    \
                        \n W_clip:        {self.w_clip}                    \
                        \n Epochs:        {self.epochs}                    \
                        \n Check_point_t: {self.check_point_t}             \
                        \n Ref, Bkg, Sig: {self.ref} {self.bkg} {self.sig} \n'
             )
        
        return  
    
    def fetch_file(self) -> str:
        '''genero il nome completo del file contenente il t finale'''
        
        self.tfile = f'1D_patience{self.check_point_t}_ref{self.ref}_bkg{self.bkg}_sig{self.sig}\
        _epochs{self.epochs}_latent{self.latent}_layers{self.layers}_wclip{self.w_clip}\
        /1D_patience{self.check_point_t}_ref{self.ref}_bkg{self.bkg}_sig{self.sig}_toy{self.toys}_t.txt'
        
        self.tfile = self.tfile.replace(' ', '')
        
        return self.tfile
    
    def fetch_history(self) -> str:
        '''genero il nome completo del file contenente il t per ogni checkpoint'''
        
        self.thistory = f'1D_patience{self.check_point_t}_ref{self.ref}_bkg{self.bkg}_sig{self.sig}\
        _epochs{self.epochs}_latent{self.latent}_layers{self.layers}_wclip{self.w_clip}\
        /1D_patience{self.check_point_t}_ref{self.ref}_bkg{self.bkg}_sig{self.sig}_toy{self.toys}_history{self.check_point_t}.h5'
        
        self.thistory = self.thistory.replace(' ', '')
        
        return self.thistory
    
    def fetch_model(self):
        '''genero il nome completo del file contenente il modello'''
        r = random.randrange(self.toys)
        self.model = [name for name in os.listdir(self.out_dir+f'/{r}/'+self.folder_name) if '_model.json' in name][0]
        self.weights = [name for name in os.listdir(self.out_dir+f'/{r}/'+self.folder_name) if '_weights.h5' in name][0]
        
        model_path = self.out_dir+f'/{r}/'+self.folder_name+'/'+self.model
        weights_path = self.out_dir+f'/{r}/'+self.folder_name+'/'+self.weights
        
        return model_path, weights_path

In [52]:
def Loss(yTrue, yPred):
    return K.sum(-1.*yTrue*(yPred) + (1-yTrue)*N_D/float(N_R)*(K.exp(yPred)-1))

In [53]:
def BuildSample(FILE_NAME, n):
     
    df=pd.read_csv(FILE_NAME)

    print('Building pandas!')
    
    df=df.sample(n=n)
    
    return df

In [85]:
WCLIP = 7 # MODIFY THIS FOR CHANGING DIRECTORY
NTOYS = 100

OUT_PATH = f'E200kW{WCLIP}'
DOF = 10
NFOLDER = 0

rPar = RunParameters(OUT_PATH, NFOLDER, NTOYS)
toys, w_clip, epochs, check_point_t, ref, bkg, sig, latent, layers = rPar.fetch_parameters()
rPar.print_parameters()

model, weights = rPar.fetch_model()


Folder name: 1D_patience1000_ref40000_bkg4000_sig0_epochs200000_latent3_layers1_wclip7.0
File name: 1D_patience1000_ref40000_bkg4000_sig0_toy100_t.txt

Parameters:                                                                      
 Toys:          100                                              
 Latent space:  3                                            
 Layers:        1                                            
 W_clip:        7.0                                            
 Epochs:        200000                                            
 Check_point_t: 1000                                     
 Ref, Bkg, Sig: 40000 4000 0 



In [86]:
SIG_NAME = '/lustre/cmswork/nlai/NPL_CODE/1D/TEST/sig.txt'
BKG_NAME = '/lustre/cmswork/nlai/NPL_CODE/1D/TEST/bkg.txt'
DATA_NAME = '/lustre/cmswork/nlai/NPL_CODE/1D/TEST/sig_bkg.txt'
REF_NAME = '/lustre/cmswork/nlai/NPL_CODE/1D/TEST/ref.txt'

N_Sig = 2000
N_Bkg = 10000
N_Ref = 40000

N_D = N_Sig + N_Bkg
N_R = N_Ref

# SIGNAL POISSON FLUCTUATIONS
# N_Sig_P = np.random.poisson(lam=N_Sig, size=1)
# N_Sig_p = N_Sig_P[0]
# print('N_Sig: '+str(N_Sig))
# print('N_Sig_Pois: '+str(N_Sig_p))

# # BACKGROUND POISSON FLUCTUATIONS
# N_Bkg_P = np.random.poisson(lam=N_Bkg, size=1)
# N_Bkg_p = N_Bkg_P[0]
# print('N_Bkg: '+str(N_Bkg))
# print('N_Bkg_Pois: '+str(N_Bkg_p))


# REFERENCE
HLF_REF = BuildSample(FILE_NAME=REF_NAME, n=N_Ref)
print('Reference DF shape:')
print(HLF_REF.shape)

# # BACKGROUND
# HLF_BKG = BuildSample(FILE_NAME=BKG_NAME, n=N_Bkg)
# print('Background DF shape:')
# print(HLF_REF.shape)

# # SIGNAL
# HLF_SIG = BuildSample(FILE_NAME=SIG_NAME, n=N_Sig)
# print('Signal DF shape:')
# print(HLF_SIG.shape)

# DATA
HLF_DATA = BuildSample(FILE_NAME=DATA_NAME, n=N_Sig+N_Bkg)
print('Data DF shape:')
print(HLF_DATA.shape)

#TARGETS
target_REF = np.zeros(N_Ref)
target_DATA = np.ones(N_Sig+N_Bkg)
target = np.append(target_REF, target_DATA)
target = np.expand_dims(target, axis=1)
print('target shape:')
print(target.shape)

feature = np.concatenate((HLF_REF, HLF_DATA), axis=0)
feature = np.concatenate((feature, target), axis=1)
print('feature shape:')
print(feature.shape)

np.random.shuffle(feature)
print(f'feature shape: {feature.shape}')
target = feature[:, -1]
feature = feature[:, :-1]

for j in range(feature.shape[1]):
    vec = feature[:, j]
    mean = np.mean(vec)
    std = np.std(vec)
    if np.min(vec) < 0:
        vec = vec-mean
        vec = vec/std
    elif np.max(vec) > 1.0:
        vec = vec *1./ mean
    feature[:, j] = vec
    
batch_size=feature.shape[0]

Building pandas!
Reference DF shape:
(40000, 1)
Building pandas!
Data DF shape:
(12000, 1)
target shape:
(52000, 1)
feature shape:
(52000, 2)
feature shape: (52000, 2)


In [87]:
# load json and create model
json_file = open(model, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(weights)
# print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss = Loss,  optimizer = 'adam')
score = loaded_model.evaluate(feature, target, batch_size=batch_size, verbose=0)

print('Final t_OBS = '+format(-2*score,'1.5f'))

Final t_OBS = -345.65863
